<a href="https://colab.research.google.com/github/Zee1988/forAIDevelop/blob/main/function_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/langchain/langhain-api-zee

/content/drive/MyDrive/langchain/langhain-api-zee


In [4]:
!pwd

/content/drive/MyDrive/langchain/langhain-api-zee


In [5]:
!ls

 agents    data_generation				     loaders
 bots	   function_call				     memory
 chains    getting_started				     prompt
 chatapi  'LangChain Cookbook Part 1 - Fundamentals.ipynb'   README.md
 data	  'LangChain Cookbook Part 2 - Use Cases.ipynb'      requirements.txt


In [6]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 10.3

In [2]:
!pwd

/content


In [3]:
%cd /content/drive/MyDrive/langchain/langhain-api-zee

/content/drive/MyDrive/langchain/langhain-api-zee


In [4]:
!pwd

/content/drive/MyDrive/langchain/langhain-api-zee


In [5]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
#openai.Model = os.environ['FAST_LLM_MODEL']
print(openai.Model)

<class 'openai.api_resources.model.Model'>


In [6]:
import openai
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

# Step 1, send model the user query and what functions it has access to
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]
    #print(message)

    # Step 2, check if the model wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        function_args = json.loads(message["function_call"]["arguments"])

        # Step 3, call the function
        # Note: the JSON response from the model may not be valid JSON
        function_response = get_current_weather(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4, send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What is the weather like in boston?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

print(run_conversation())

{
  "id": "chatcmpl-7ST1WYKPnU4ravcvkKcBNrReh2DXq",
  "object": "chat.completion",
  "created": 1687019106,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in Boston is currently sunny and windy with a temperature of 72 degrees Fahrenheit."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 73,
    "completion_tokens": 18,
    "total_tokens": 91
  }
}


In [7]:
def getName(name):
    """Get the English name in a given name"""
    name_info = {
        "shanghui":"Zee",
        "gonghai":"Randy",
        "zhongling":"Will",
        "qiufeng":"Coco"
    }

    return name_info[name]

def run_conversation():
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo-0613",
        messages = [{"role":"user","content":"What's the English name of shanghui?"}],
        functions = [
            {
                "name" : "getName",
                "description" : "Get the English name in a given name",
                "parameters":{
                    "type":"object",
                    "properties":{
                        "name":{
                            "type":"string",
                            "description":"The name related to the English name."
                        }
                    },
                    "required": ["name"],
                },
            }
        ],
         function_call="auto",
    )

    message = response["choices"][0]["message"]
    print(message)

    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        function_args = json.loads(message["function_call"]["arguments"])
        function_response = getName(
            name=function_args.get("name"),
        )
        print(function_response)

    second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What's the English name of shanghui?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
    return second_response["choices"][0]["message"]


print(run_conversation())

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "getName",
    "arguments": "{\n  \"name\": \"shanghui\"\n}"
  }
}
Zee
{
  "role": "assistant",
  "content": "The English name for \"shanghui\" is Zee."
}
